# ПРОШУ ПРОВЕРИТЬ МОЙ ПРОЕКТ МАКСИМАЛЬНО СТРОГО

_____
_____
## Описание проекта

_____
_____
## Импорт и подготовка к работе

In [1]:
# %%capture
# %pip install pymystem3
# %pip install spacy
# %pip install --upgrade ipywidgets
# %pip install transformers[torch] datasets scikit-learn torch

In [2]:
import time
import re
import os
import spacy
import pandas as pd
import spacy.cli
from tqdm.notebook import tqdm
import requests
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.compose import ColumnTransformer
from plotly import express as px, graph_objects as go
import numpy as np
from typing import Literal
from sklearn.pipeline import Pipeline
from pprint import pprint
from optuna.visualization import plot_param_importances
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.feature_selection import (
    RFE,
    SelectKBest,
    mutual_info_regression
)
from tqdm.notebook import tqdm
import torch
from transformers import BertModel, BertTokenizer

from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.base import clone
from copy import deepcopy
from sklearn.model_selection import cross_val_score, KFold
import optuna
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from itertools import cycle

In [3]:
tqdm.pandas()
spacy.cli.download('en_core_web_sm')
RANDOM_STATE = 42
optuna_sampler = optuna.samplers.TPESampler(seed=RANDOM_STATE)
state = np.random.RandomState(RANDOM_STATE)
cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
color_palette = cycle(px.colors.qualitative.Plotly)
optuna_storage = 'sqlite:///optuna.db'
stop_words = list(set(stopwords.words('english')))


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


_____
_____
## Загрузка данных, общая информация и первичная обработка

|Поле      |Описание                                       |
|----------|-----------------------------------------------|
|Unnamed: 0|Индекс                                         |
|text      |Текст пользовательского комментария            |
|toxic     |Метка токсичности комментария (0 — нет, 1 — да)|


In [4]:
path_1 = 'toxic_comments.csv'
path_2 = '/datasets/toxic_comments.csv'
if os.path.exists(path_1):
    comments = pd.read_csv(path_1, delimiter=',', parse_dates=[0])
elif os.path.exists(path_2):
    comments = pd.read_csv(path_2, delimiter=',', parse_dates=[0])
else:
    raise FileNotFoundError

C:\Users\bezgr\AppData\Local\Temp\ipykernel_3196\579967484.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  comments = pd.read_csv(path_1, delimiter=',', parse_dates=[0])


In [5]:
display(comments.head())
comments.info()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  object
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 3.6+ MB


_____
### Вывод:

- Таблица содержит тексты пользовательских комментариев и метки их токсичности.
- Всего 159 292 строки.
- Пропущенных значений нет.
- 1 количественный признак: `toxic`.
- 2 категориальных признака: `Unnamed: 0` и `text`.
- Необходима предобработка текста

_____
_____
## Предобработка данных

_____
### Переименовывание столбцов датафреймов

*Удалим столбец `Unnamed: 0`, а также создадим словарь с переводом*

In [6]:
comments = comments.drop('Unnamed: 0', axis=1)

column_translation = {
    'text': 'Текст комментария',
    'toxic': 'Токсичность'
}

_____
### Удаление дубликатов и опечаток

In [7]:
print('Кол-во дубликатов в comments:',
      comments.duplicated().sum() + comments.text.duplicated().sum())

Кол-во дубликатов в comments: 0


_____
### Очистка и лемматизация текста

In [ ]:
RE_CLEAN = re.compile(r'[^a-zA-Z ]')
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])


def clean_text(text: str) -> str:
    '''Удаляет лишние символы, приводит к нижнему регистру и убирает лишние пробелы'''
    return ' '.join(RE_CLEAN.sub(' ', str(text).lower()).split())


def lemmatize_corpus(texts: pd.Series) -> pd.Series:
    '''Лемматизирует тексты с отображением прогресса'''
    texts = texts.dropna().astype(str)
    lemmatized = (
        ' '.join(token.lemma_ for token in doc)
        for doc in tqdm(
            nlp.pipe(texts, batch_size=100, n_process=os.cpu_count()-1),
            total=len(texts),
            desc='Lemmatizing'
        )
    )
    return pd.Series(lemmatized, index=texts.index)

*Подкачиваем чекпоинт*

In [ ]:
path_csv = 'prepared_data/lemmatized_text.csv'
if os.path.exists(path_csv):
    comments = pd.read_csv(path_csv)
else:
    comments['cleared_text'] = comments.text.apply(clean_text)
    comments['lemmatized_text'] = lemmatize_corpus(comments.cleared_text)
    comments.drop('text', axis=1).to_csv(path_csv, index=False)
column_translation |= {
    'cleared_text': 'очищенный текст',
    'lemmatized_text': 'лемматизированный текст'
}

In [10]:
comments

,toxic,cleared_text,lemmatized_text
0,0,explanation why the edits made under my userna...,explanation why the edit make under my usernam...
1,0,d aww he matches this background colour i m se...,d aww he match this background colour I m seem...
2,0,hey man i m really not trying to edit war it s...,hey man I m really not try to edit war it s ju...
3,0,more i can t make any real suggestions on impr...,more I can t make any real suggestion on impro...
4,0,you sir are my hero any chance you remember wh...,you sir be my hero any chance you remember wha...
...,...,...,...
159287,0,and for the second time of asking when your vi...,and for the second time of ask when your view ...
159288,0,you should be ashamed of yourself that is a ho...,you should be ashamed of yourself that be a ho...
159289,0,spitzer umm theres no actual article for prost...,spitzer umm there s no actual article for pros...
159290,0,and it looks like it was actually you who put ...,and it look like it be actually you who put on...


_____
### Удаление пропущенных значений

*после очистки появились пропуски*

In [11]:
comments = comments[~comments.isna().any(axis=1)]

_____
### Вывод:

- Удалён неинформативный столбец Unnamed: 0.
- Проведена очистка текста: удалены лишние символы, приведено к нижнему регистру, устранены лишние пробелы.
- Выполнена лемматизация текстов с использованием модели spacy.
- После предобработки удалены строки с пропущенными значениями.

_____
_____
## Анализ данных

_____
### Функции отрисовок данных

In [ ]:
def statistical_graphis_for_categorical(data: pd.Series, agg: Literal['val_count', 'sum'] = 'val_count', top_n_in_bar=20, top_n_in_pie=5):
    '''
    Функция для построения графиков по категориальным данным: столбчатые диаграммы и круговой диаграммы.
    
    Parameters
    ----------
    data : pd.Series
        Входные данные в виде столбца (серии) pandas, представляющие категориальную переменную.
        
    agg : {'val_count', 'sum'}, по умолчанию 'val_count'
        Метод агрегации для категориальных данных:
        - 'val_count' — подсчитывает количество наблюдений в каждой категории.
        - 'sum' — суммирует значения в каждой категории (например, для числовых категорий).
        
    top_n_in_bar : int, по умолчанию 20
        Количество топ-N категорий, которые будут отображены в столбчатой диаграмме.
        Остальные категории будут объединены в одну категорию 'Остальные'.
    
    top_n_in_pie : int, по умолчанию 5
        Количество топ-N категорий, которые будут отображены на круговой диаграмме.
        Остальные категории будут объединены в одну категорию 'Остальные'.
    '''
    data = data.sample(3000, random_state=state)
    fig = make_subplots(
        rows=1, cols=2, specs=[[{'type': 'xy'}, {'type': 'domain'}]],
        subplot_titles=('Гистограмма', 'Круговая диаграмма')
    )
    
    if agg == 'val_count':
        category_agg = data.value_counts()
    elif agg == 'sum':
        category_agg = data.groupby(data).sum()
    else:
        raise ValueError('agg="val_count"|"sum"')
    
    # Обработка для top_n_in_bar
    if len(category_agg) > top_n_in_bar:
        other_value = category_agg[top_n_in_bar:].sum()
        category_agg = pd.concat([category_agg.head(top_n_in_bar), pd.Series({'Остальные': other_value})])
    
    categories = category_agg.index
    val_agg = category_agg.values
    
    fig.add_trace(
        go.Bar(
            x=val_agg, y=categories, 
            orientation='h',
            marker_color='green', 
            showlegend=False, 
            name=data.name,
            width=0.8
        ),
        row=1, col=1
    )
    fig.update_xaxes(title_text='Частота' if agg == 'val_count' else 'Сумма', row=1, col=1)
    fig.update_yaxes(title_text=column_translation.get(data.name, data.name), tickvals=categories, row=1, col=1)
    
    # Обработка для top_n_in_pie
    other_pie = [category_agg[top_n_in_pie:].sum()]
    fig.add_trace(
        go.Pie(labels=category_agg.head(top_n_in_pie).index.tolist() + (['Остальные'] if other_pie[0] else []),
               values=category_agg.head(top_n_in_pie).values.tolist() + (other_pie if other_pie[0] else []),
               name='',
               textinfo='label+percent'),
        row=1, col=2
    )
    
    fig.update_layout(
        title_text=f'Статистические графики по колонке <b>{column_translation.get(data.name, data.name)}</b><br>(<b>{data.name}</b>)',
        title_x=0.5,
        showlegend=True,
        width=1200,
        height=600,
    )
    
    fig.show()

def statistical_graphis_for_numeric(data: pd.Series, title_text=None, nbinsx=50):
    '''
    Функция для построения графиков для числовых данных: гистограммы и диаграммы размаха.
    
    Parameters
    ----------
    data : pd.Series
        Входные данные в виде столбца (серии) pandas, представляющие числовую переменную.
        
    nbinsx : int, по умолчанию 50
        Количество корзин (bins) для построения гистограммы. Управляет точностью распределения данных по оси x.
    '''
    data = data.sample(3000, random_state=state)
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Гистограмма', 'Диаграмма размаха'))
    
    fig.add_trace(
        go.Histogram(x=data, nbinsx=nbinsx, marker_color='blue', name=data.name),
        row=1, col=1
    )
    fig.update_xaxes(title_text=column_translation[data.name], row=1, col=1)
    fig.update_yaxes(title_text='Частота', row=1, col=1)

    fig.add_trace(
        go.Box(y=data, marker_color='orange', name=''),
        row=1, col=2
    )
    fig.update_yaxes(title_text=column_translation[data.name], row=1, col=2)
    fig.update_layout(
        title_text=(title_text if title_text
                    else f'Статистические графики по значению <b>{column_translation[data.name]}</b><br>(<b>{data.name}</b>)'),
        title_x=0.5,
        showlegend=False,
        width=1200,
        height=500
    )

    fig.show()


-----

### Таргет (`toxic`)

In [13]:
statistical_graphis_for_categorical(comments.toxic)

### Длина комментариев (`cleared_text`)

In [14]:
statistical_graphis_for_numeric(comments.cleared_text.str.split().str.len(),
                                title_text='Распределение кол-ва слов в твитах',
                                nbinsx=70)

_____
### Вывод:

- Распределение целевой переменной toxic является дисбалансированным.
- Распределение длины очищенных комментариев (cleared_text) имеет сильную асимметрию: большая часть текстов содержит до 50 слов, но встречаются выбросы — тексты длиной более 1000 слов.
- Наличие выбросов подтверждается на диаграмме размаха, где видно значительное количество экстремально длинных комментариев.
- Такие особенности данных могут повлиять на обучение моделей и требуют дополнительного внимания (например, можно ограничить максимальную длину текста при токенизации).

_____
_____
## Модели

_____
### Оценка моделей

In [15]:
def trial_duration_performance(*studies: optuna.Study,
                               model_names: list[str],
                               score_name='F1',
                               greater_is_better=True):
    """
    Визуализация зависимости метрики качества от времени выполнения трейла для одного или нескольких исследований Optuna.

    Parameters
    ----------
    studies : optuna.Study
        Одно или несколько объектов Optuna Study, содержащих результаты гиперпараметрического поиска.

    model_names : list of str
        Список имён моделей, соответствующий каждому исследованию (в том же порядке).

    score_name : str, default='F1'
        Название метрики, используемой в исследовании (отображается на оси Y и в заголовках).

    greater_is_better : bool, default=True
        Флаг, указывающий, нужно ли максимизировать метрику (True) или минимизировать (False).
    """
    fig = go.Figure()

    for i, study in enumerate(studies):
        color = color_palette.__next__()
        df = study.trials_dataframe()
        df = df[df.value.notnull()].copy()
        df['duration_sec'] = df['duration'].dt.total_seconds()
        df = df.sort_values(by=['duration_sec', 'value'], ascending=[True, False])

        fig.add_trace(go.Scatter(
            x=df['duration_sec'],
            y=df['value'],
            mode='markers',
            marker=dict(color=color, opacity=0.5, size=10),
            name=model_names[i],
            legendgroup=str(i),
            legendgrouptitle=dict(text=model_names[i])
        ))

        best_idx = df['value'].idxmax() if greater_is_better else df['value'].idxmin()
        best_point = df.loc[best_idx]

        fig.add_trace(go.Scatter(
            x=[best_point['duration_sec']],
            y=[best_point['value']],
            mode='markers',
            marker=dict(color=color, size=10, line=dict(color='black', width=2)),
            showlegend=False,
            name=f'Лучшее: {model_names[i]}',
            legendgroup=str(i),
        ))

    fig.update_layout(
        title_text=f'{score_name} в зависимости от времени трейла',
        xaxis_title='Время выполнения трейла (сек)',
        yaxis_title=score_name,
        showlegend=True,
        template='plotly_white',
        width=800,
        height=500
    )
    fig.show()


def trial_score_distribution(*studies: optuna.Study,
                             model_names: list[str],
                             nbinsx: dict[str, int] = None,
                             xrange: tuple[float, float] = None,
                             score_name='F1'):
    """
    Визуализация распределения значений метрики качества для одного или нескольких исследований Optuna.

    Parameters
    ----------
    studies : optuna.Study
        Одно или несколько объектов Optuna Study, содержащих трейлы с оценками моделей.

    model_names : list of str
        Список имён моделей, соответствующий каждому исследованию (в том же порядке).

    nbinsx : dict[str, int], optional
        Словарь, в котором ключ — имя модели, а значение — количество корзин (bins) для гистограммы.
        Если не указан, будет использовано значение по умолчанию от Plotly.

    xrange : tuple of float, optional
        Кортеж вида (min, max), задающий диапазон оси X (метрики).
        Полезно при сравнении моделей с разным масштабом значений.

    score_name : str, default='F1'
        Название метрики, отображаемое на графиках.
    """

    fig = go.Figure()

    for i, study in enumerate(studies):
        color = color_palette.__next__()
        df = study.trials_dataframe()
        df = df[df.value.notnull()].copy()

        fig.add_trace(go.Histogram(
            x=df['value'],
            nbinsx=nbinsx[model_names[i]] if nbinsx else None,
            name=model_names[i],
            marker_color=color,
            opacity=0.6,
            legendgroup=str(i),
            legendgrouptitle=dict(text=model_names[i])
        ))

    fig.update_layout(
        title_text=f'Распределение значений метрики {score_name}',
        xaxis_title=score_name,
        yaxis_title='Количество трейлов',
        barmode='overlay',
        template='plotly_white',
        showlegend=True,
        width=800,
        height=500,
        xaxis=dict(range=xrange)
    )
    fig.show()


_____
### Пайплайн

In [16]:
def get_pipepline(lemm_column: str,
                  feature_selection: Literal['RFE', 'SelectKBest', None] = None) -> Pipeline:
    """
    Функция для создания пайплайна классификации с различными методами предварительной обработки данных и опциональными методами отбора признаков.
    
    В зависимости от выбранного метода отбора признаков (RFE или SelectKBest) и предоставленных данных, функция создает пайплайн,
    который выполняет предварительную обработку числовых и категориальных данных, а также может включать этап отбора признаков.

    Parameters
    ----------
    columns : list of str
        Список названий признаков.

    feature_selection : {'RFE', 'SelectKBest', None}, по умолчанию None
        Метод отбора признаков, который будет использоваться в пайплайне:
        - 'RFE' — применяет рекурсивный отбор признаков (RFE).
        - 'SelectKBest' — применяет метод выбора K лучших признаков с использованием статистики mutual_info_regression или f_classif.
        - None — без отбора признаков.

    Returns
    -------
    Pipeline
        Возвращает объект `Pipeline`, который включает в себя этапы предварительной обработки и, опционально, этап отбора признаков.
    """
    preprocessor = ColumnTransformer(
        [('vectorizer', 'passthrough', lemm_column)],
        verbose_feature_names_out=True,
    )
    if feature_selection == 'RFE':
        model = RFE(estimator=DummyClassifier(),
                    step=1,
                    verbose=0)
        return Pipeline(
            steps=[
                ('preprocessor', preprocessor),
                ('model', model)
            ]
        )
    elif feature_selection == 'SelectKBest':
        return Pipeline(
            steps=[
                ('preprocessor', preprocessor),
                ('feature_selection', SelectKBest(
                    score_func=mutual_info_regression
                )),
                ('model', DummyClassifier())
            ]
        )
    else:
        return Pipeline(
            steps=[
                ('preprocessor', preprocessor),
                ('model', DummyClassifier())
            ]
        )

_____
### Датасеты

In [17]:
X = comments[['lemmatized_text']]
y = comments.toxic
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_STATE, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, random_state=RANDOM_STATE, stratify=y_test)

_____
### Перебор гиперпараметров через optuna

*Перебираемые параметры*

In [18]:
preprocessor_params = {
    'preprocessor__vectorizer':
        [CountVectorizer(stop_words=list(stop_words), dtype=np.float32),
         TfidfVectorizer(stop_words = list(stop_words), dtype=np.float32)]
}
model_params_cat = {}
model_params_num = {}
pipeline = None

*Функция для умного перебора гиперпарметров через optuna*

In [19]:
def get_named_params(best_params):
    cat_params = {**preprocessor_params, **model_params_cat}
    return {
        param: cat_params[param][best_params[param]] if param in cat_params
               else best_params[param]
        for param in best_params
    }


def objective_reg(trial : optuna.trial.Trial) -> float:
    """
    Целевая функция для оптимизации гиперпараметров с использованием библиотеки Optuna.
    В этой функции выполняется настройка и обучение модели с использованием предложенных значений гиперпараметров,
    а затем оценивается её точность на валидационной выборке.

    Parameters
    ----------
    trial : optuna.trial.Trial
        Экземпляр объекта `Trial` из библиотеки Optuna, который используется для выбора гиперпараметров.

    Returns
    -------
    float
        Оценка модели (например, точность) на валидационной выборке. Если возникает ошибка, пробный эксперимент прерывается.
    """
    categorical_params = {**deepcopy(preprocessor_params), **deepcopy(model_params_cat)}
    params_cat = {
        param_name: categorical_params[param_name][trial.suggest_categorical(param_name, range(0, len(categorical_params[param_name])))]
        for param_name in categorical_params
    }
    params_num = {param_name: model_params_num[param_name](trial) for param_name in model_params_num}
    pipeline_temp:Pipeline = clone(pipeline)
    pipeline_temp.set_params(**params_cat, **params_num)
    score = cross_val_score(pipeline_temp, X_train, y_train, cv=cv, scoring='f1', n_jobs=5).mean()
    trial.report(score, step=1)
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()
    return score

❗️Примечание - поиск гиперпараметров работает с [чекпоинтами](https://github.com/GrishaTS/Ya-Practicum-DS/blob/main/10.%20%D0%A7%D0%B8%D1%81%D0%BB%D0%B5%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D1%8B/optuna.db). Можно скачать и не тратить время на повторное обучение.

In [20]:
response = requests.get(
    'https://github.com/GrishaTS/Ya-Practicum-DS/raw/main/12.%20Машинное%20обучение%20для%20текстов/optuna.db',
    stream=True
)
if response.status_code == 200:
    with open('optuna.db', 'wb') as f:
        for chunk in response.iter_content(1024):
            f.write(chunk)
    print('Файл успешно скачан: optuna.db')
else:
    print('Ошибка загрузки:', response.status_code)


Файл успешно скачан: optuna.db


In [21]:
def get_trained_optuna_study(study_name, restudy=False, direction='minimize', n_trials=100):
    """
    Создаёт или загружает существующее исследование Optuna для оптимизации гиперпараметров.

    Функция загружает существующее исследование Optuna по заданному имени, если оно уже создано.
    Если параметр `restudy=True`, удаляет предыдущее исследование и создаёт новое.
    При создании нового исследования выполняется оптимизация заданной целевой функции.

    Parameters
    ----------
    study_name : str
        Имя исследования в Optuna.
    restudy : bool, optional, default=False
        Если `True`, удаляет существующее исследование с таким же именем перед созданием нового.
    direction : {'minimize', 'maximize'}, optional, default='minimize'
        Направление оптимизации: `minimize` для минимизации функции потерь, `maximize` для её максимизации.
    n_trials : int, optional, default=100
        Количество итераций (экспериментов) для оптимизации гиперпараметров.

    Returns
    -------
    optuna.study.Study
        Объект `Study`, содержащий результаты оптимизации гиперпараметров.
    """
    study_names = optuna.study.get_all_study_names(storage=optuna_storage)
    if study_name in study_names and not restudy:
        study = optuna.load_study(storage=optuna_storage, study_name=study_name)
    else:
        if study_name in study_names:
            optuna.delete_study(study_name=study_name, storage=optuna_storage)
        study = optuna.create_study(study_name=study_name,
                                    direction=direction,
                                    sampler=optuna_sampler,
                                    storage=optuna_storage)
        study.optimize(objective_reg, n_trials=n_trials, show_progress_bar=True)
    return study

_____
### LogisticRegression

In [22]:
pipeline = get_pipepline(
    lemm_column=X.columns[0],
)
model_params_cat = {
    'model': [LogisticRegression(n_jobs=2)],
    'preprocessor__vectorizer__ngram_range': [(1, 1), (1, 2)],
}
model_params_num = {
    'model__C': lambda trial: trial.suggest_float('model__C', 0.1, 10, log=True),
    'model__max_iter': lambda trial: trial.suggest_int('model__max_iter', 100, 300, step=50),
}

In [23]:
study_logreg = get_trained_optuna_study('logreg', restudy=False, direction='maximize')

In [24]:
print(f'Best cross-val f1: {study_logreg.best_value:.3f}')
best_params_logreg = get_named_params(study_logreg.best_params)
print('\nBest parameters:')
pprint(best_params_logreg)

Best cross-val f1: 0.779

Best parameters:
{'model': LogisticRegression(n_jobs=2),
 'model__C': 9.986424465040107,
 'model__max_iter': 200,
 'preprocessor__vectorizer': CountVectorizer(dtype=<class 'numpy.float32'>,
                stop_words=["that'll", 'while', 'ain', 'between', "mightn't",
                            'm', 'her', 'where', 'just', 'their', 'am',
                            "didn't", "hadn't", "they've", 'shouldn', 'didn',
                            'i', 've', "wouldn't", 'for', "we'd", 'itself',
                            "you've", 'won', 'those', 'from', 'all',
                            "shouldn't", 'to', 'mightn', ...]),
 'preprocessor__vectorizer__ngram_range': (1, 2)}


In [25]:
plot_param_importances(study_logreg).update_layout(width=800, height=500).show()
trial_duration_performance(study_logreg, model_names=['LogisticRegression'])
trial_score_distribution(study_logreg,
                         model_names=['LogisticRegression'],
                         nbinsx={'LogisticRegression': 300},
                         xrange=(0.725, 0.78))

- Важность гиперпараметров
    - Наибольшее влияние на качество модели оказывает параметр model__C (вес регуляризации), его важность составляет 0.69.
    - Далее по значимости — preprocessor__vectorizer (тип векторизации), с важностью 0.26.
- Зависимость F1 от времени выполнения трейла
    - На графике зависимости F1 от времени трейла видно, что модель достаточно стабильна: большинство трейлов дают результат выше 0.76, вне зависимости от времени выполнения.
    - Лучшее значение F1 ≈ 0.78, при сравнительно небольшом времени работы (< 100 секунд).
    - Это указывает на эффективность Logistic Regression — хорошее качество при малых затратах ресурсов.
- Распределение значений метрики F1
    - Распределение значений F1 метрики имеет сильный сдвиг вправо, большая часть значений находится в диапазоне 0.77–0.78.
    - Это говорит о том, что большинство конфигураций гиперпараметров дают сходное, высокое качество, а значит модель устойчива к выбору гиперпараметров.

_____
### PassiveAggressiveClassifier

In [26]:
pipeline = get_pipepline(
    lemm_column=X.columns[0],
)
model_params_cat = {
    'model': [PassiveAggressiveClassifier(n_jobs=2)],
    'model__loss': ['hinge', 'squared_hinge'],
    'preprocessor__vectorizer__ngram_range': [(1, 1), (1, 2)],
}
model_params_num = {
    'model__C': lambda trial: trial.suggest_float('model__C', 0.1, 10, log=True),
    'model__max_iter': lambda trial: trial.suggest_int('model__max_iter', 100, 300, step=50),
}

In [27]:
study_palogreg = get_trained_optuna_study('palogreg', restudy=False, direction='maximize')

In [28]:
print(f'Best cross-val f1: {study_palogreg.best_value:.3f}')
best_params_palogreg = get_named_params(study_palogreg.best_params)
print('\nBest parameters:')
pprint(best_params_palogreg)

Best cross-val f1: 0.794

Best parameters:
{'model': PassiveAggressiveClassifier(n_jobs=2),
 'model__C': 0.12050679650559393,
 'model__loss': 'hinge',
 'model__max_iter': 300,
 'preprocessor__vectorizer': TfidfVectorizer(dtype=<class 'numpy.float32'>,
                stop_words=["that'll", 'while', 'ain', 'between', "mightn't",
                            'm', 'her', 'where', 'just', 'their', 'am',
                            "didn't", "hadn't", "they've", 'shouldn', 'didn',
                            'i', 've', "wouldn't", 'for', "we'd", 'itself',
                            "you've", 'won', 'those', 'from', 'all',
                            "shouldn't", 'to', 'mightn', ...]),
 'preprocessor__vectorizer__ngram_range': (1, 2)}


In [29]:
plot_param_importances(study_palogreg).update_layout(width=800, height=500).show()
trial_duration_performance(study_palogreg, model_names=['PassiveAggressiveClassifier'])
trial_score_distribution(study_palogreg,
                         model_names=['PassiveAggressiveClassifier'],
                         nbinsx={'PassiveAggressiveClassifier': 25},
                         xrange=(0.73, 0.795))

- Важность гиперпараметров
    - Наибольшее влияние на качество модели оказывает параметр preprocessor__vectorizer, его важность составляет 0.61.
    - Вторым по значимости идёт preprocessor__vectorizer__ngram_range с важностью 0.34.
- Зависимость F1 от времени выполнения трейла
    - Модель демонстрирует очень быструю скорость работы: почти все трейлы выполняются за 17–19 секунд.
    - Значения F1 концентрируются в верхней части шкалы, большинство трейлов дают результат 0.78–0.795.
    - Лучшее значение F1 достигает примерно 0.796, при этом затраты по времени остаются минимальными.
    - Это делает PassiveAggressiveClassifier отличным выбором для быстрых итераций, особенно в задачах с ограниченным временем обучения.
- Распределение значений метрики F1
    - Распределение F1 метрики имеет выраженный пик около 0.79, что свидетельствует о высокой устойчивости качества модели.
    - Основная масса трейлов показывает высокие значения метрики, выше 0.78, что говорит о стабильной и предсказуемой работе модели в пределах гиперпараметрического поиска.

_____
### RandomForestClassifier

In [30]:
pipeline = get_pipepline(
    lemm_column=X.columns[0],
)
model_params_cat = {
    'model': [RandomForestClassifier(n_jobs=2)],
    'preprocessor__vectorizer__ngram_range': [(1, 1), (1, 2)],
}
model_params_num = {
    'model__n_estimators': lambda trial: trial.suggest_int('model__n_estimators', 50, 100),
    'model__max_samples': lambda trial: trial.suggest_float('model__max_samples', 0.5, 0.7),
    'model__max_depth': lambda trial: trial.suggest_int('model__max_depth', 7, 15),
    'model__min_samples_split': lambda trial: trial.suggest_int('model__min_samples_split', 3, 7),
    'model__min_samples_leaf': lambda trial: trial.suggest_int('model__min_samples_leaf', 2, 4),
}

In [31]:
study_randomforest = get_trained_optuna_study('randomforest', restudy=False, direction='maximize')

In [32]:
print(f'Best cross-val f1: {study_randomforest.best_value:.3f}')
best_params_randomforest = get_named_params(study_randomforest.best_params)
print('\nBest parameters:')
pprint(best_params_randomforest)

Best cross-val f1: 0.001

Best parameters:
{'model': RandomForestClassifier(n_jobs=2),
 'model__max_depth': 15,
 'model__max_samples': 0.6365582818549287,
 'model__min_samples_leaf': 2,
 'model__min_samples_split': 3,
 'model__n_estimators': 55,
 'preprocessor__vectorizer': CountVectorizer(dtype=<class 'numpy.float32'>,
                stop_words=["that'll", 'while', 'ain', 'between', "mightn't",
                            'm', 'her', 'where', 'just', 'their', 'am',
                            "didn't", "hadn't", "they've", 'shouldn', 'didn',
                            'i', 've', "wouldn't", 'for', "we'd", 'itself',
                            "you've", 'won', 'those', 'from', 'all',
                            "shouldn't", 'to', 'mightn', ...]),
 'preprocessor__vectorizer__ngram_range': (1, 1)}


- Модель не показала достотчного результата, поэтому далее не рассматривается.

_____
### LGBMClassifier

In [33]:
pipeline = get_pipepline(
    lemm_column=X.columns[0],
)

model_params_cat = {
    'model': [LGBMClassifier(random_state=RANDOM_STATE, n_jobs=2, objective='rmse')],
    'model__boosting_type': ['gbdt', 'rf', 'goss'],
    'model__max_depth': [-1, 5, 10, 15, 20],
    'preprocessor__vectorizer__ngram_range': [(1, 1), (1, 2)],
}
model_params_num = {
    'model__learning_rate': lambda trial: trial.suggest_float('model__learning_rate', 0.01, 0.99),
    'model__n_estimators': lambda trial: trial.suggest_int('model__n_estimators', 80, 150),
    'model__min_child_samples': lambda trial: trial.suggest_int('model__min_child_samples', 5, 20),
    'model__subsample': lambda trial: trial.suggest_float('model__subsample', 0.5, 0.9),
    'model__colsample_bytree': lambda trial: trial.suggest_float('model__colsample_bytree', 0.6, 1),
    'model__reg_alpha': lambda trial: trial.suggest_float('model__reg_alpha', 0.00, 0.1),
    'model__reg_lambda': lambda trial: trial.suggest_float('model__reg_lambda', 0.00, 0.1),
}

In [34]:
study_lgbm = get_trained_optuna_study('lgbm', restudy=False, direction='maximize')

In [35]:
print(f'Best cross-val f1: {study_lgbm.best_value:.3f}')
best_params_lgbm = get_named_params(study_lgbm.best_params)
print('\nBest parameters:')
pprint(best_params_lgbm)

Best cross-val f1: 0.771

Best parameters:
{'model': LGBMClassifier(n_jobs=2, objective='rmse', random_state=42),
 'model__boosting_type': 'gbdt',
 'model__colsample_bytree': 0.6396857600825309,
 'model__learning_rate': 0.3886072432451956,
 'model__max_depth': -1,
 'model__min_child_samples': 16,
 'model__n_estimators': 98,
 'model__reg_alpha': 0.07713052207546063,
 'model__reg_lambda': 0.07371242795466323,
 'model__subsample': 0.7681788129204197,
 'preprocessor__vectorizer': TfidfVectorizer(dtype=<class 'numpy.float32'>,
                stop_words=["that'll", 'while', 'ain', 'between', "mightn't",
                            'm', 'her', 'where', 'just', 'their', 'am',
                            "didn't", "hadn't", "they've", 'shouldn', 'didn',
                            'i', 've', "wouldn't", 'for', "we'd", 'itself',
                            "you've", 'won', 'those', 'from', 'all',
                            "shouldn't", 'to', 'mightn', ...]),
 'preprocessor__vectorizer__ngram_r

In [36]:
plot_param_importances(study_lgbm).update_layout(width=800, height=500).show()
trial_duration_performance(study_lgbm, model_names=['LGBMClassifier'])
trial_score_distribution(study_lgbm,
                         model_names=['LGBMClassifier'],
                         nbinsx={'LGBMClassifier': 30},
                         xrange=(0.61, 0.78))

- Важность гиперпараметров
    - Наибольшее влияние на метрику оказывает параметр preprocessor__vectorizer с важностью 0.64.
    - Далее идут model__learning_rate (0.16) и model__boosting_type (0.13), что говорит о чувствительности модели к типу бустинга и скорости обучения.
- Зависимость F1 от времени выполнения трейла
    - Время выполнения трейлов заметно выше по сравнению с линейными моделями — от 50 до 200 секунд.
    - Наблюдается разброс F1-метрики — от 0.64 до 0.76, что говорит о нестабильности при разных гиперпараметрах.
    - Наилучшие результаты (F1 ≈ 0.76) достигаются при времени работы около 100–120 секунд.
- Распределение значений метрики F1
    - Распределение имеет широкий разброс, что говорит о чувствительности модели к настройке гиперпараметров.
    - Большинство значений сосредоточены в диапазоне 0.72–0.76, с пиком около 0.75.
    - Несмотря на наличие слабых результатов (ниже 0.70), значительная доля трейлов показывает высокое качество, если параметры подобраны удачно.

_____
### Результат моделей на частотных векторах

In [37]:
trial_duration_performance(
    study_logreg, study_palogreg, study_lgbm,
    model_names=['LogisticRegression',
                 'PassiveAggressiveClassifier',
                 'LGBMClassifier']
)
trial_score_distribution(
    study_logreg, study_palogreg, study_lgbm,
    model_names=['LogisticRegression',
                 'PassiveAggressiveClassifier',
                 'LGBMClassifier'],
    nbinsx={'LogisticRegression': 300,
            'PassiveAggressiveClassifier': 25,
            'LGBMClassifier': 30},
    xrange=(0.71, 0.795)
)

In [38]:
analys = pd.DataFrame(columns=['Модель', 'F1-score', 'Время обучения (с)', 'Время предсказания (мс)']).set_index('Модель', drop=True)
for model_name, params in [('LogisticRegression', best_params_logreg),
                           ('PassiveAggressiveClassifier', best_params_palogreg),
                           ('LGBMClassifier', best_params_lgbm)]:
    model = clone(pipeline).set_params(**params)
    point_0 = time.time()
    model.fit(X_train, y_train)
    point_1 = time.time()
    y_pred = model.predict(X_val)
    point_2 = time.time()
    analys.loc[model_name] = [f1_score(y_val, y_pred), point_1 - point_0, (point_2 - point_1) / X_val.shape[0] * 1000]
analys.sort_values(by=['F1-score'], ascending=False).style.background_gradient(
    'Oranges', subset=['F1-score', 'Время обучения (с)', 'Время предсказания (мс)']
).format(precision=3)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 5.855402 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 862219
[LightGBM] [Info] Number of data points in the train set: 119460, number of used features: 30096
[LightGBM] [Info] Start training from score 0.101616


,F1-score,Время обучения (с),Время предсказания (мс)
Модель,,,
PassiveAggressiveClassifier,0.784,11.455,0.054
LGBMClassifier,0.768,48.780,0.074
LogisticRegression,0.765,58.152,0.053


- Качество моделей (F1-score)
    - Наилучшее качество показала модель PassiveAggressiveClassifier — F1 = 0.785.
    - LGBMClassifier и LogisticRegression имеют сопоставимые, но более низкие значения: 0.768 и 0.765 соответственно.
    - Разница между LGBM и LogisticRegression минимальна, однако PassiveAggressiveClassifier уверенно лидирует.
- Время обучения
    - PassiveAggressiveClassifier — самый быстрый по обучению: 12.7 секунд.
    - LGBMClassifier обучается в среднем за 53.1 секунды, что значительно медленнее.
    - LogisticRegression неожиданно оказался самым медленным — 67.6 секунд.
- Время предсказания
    - Все модели демонстрируют высокую скорость предсказания (меньше 0.1 мс).
    - LogisticRegression — самый быстрый: 0.055 мс.
    - PassiveAggressiveClassifier — 0.060 мс, LGBMClassifier — 0.075 мс.

*Среди представленных моделей лучше всего показала себя PassiveAggressiveClassifier, ее и возьмем за итоговую*

In [39]:
model = clone(pipeline).set_params(**best_params_palogreg)
model.fit(pd.concat([X_train, X_val], axis=0), pd.concat([y_train, y_val], axis=0))

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('vectorizer',
                                                  TfidfVectorizer(dtype=<class 'numpy.float32'>,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  stop_words=["that'll",
                                                                              'while',
                                                                              'ain',
                                                                              'between',
                                                                              "mightn't",
                                                                              'm',
                                                                              'her',
                                                                              'where',
                                                                              'just',
                                                                              'their',
                                                                              'am',
                                                                              "didn't",
                                                                              "hadn't",
                                                                              "they've",
                                                                              'shouldn',
                                                                              'didn',
                                                                              'i',
                                                                              've',
                                                                              "wouldn't",
                                                                              'for',
                                                                              "we'd",
                                                                              'itself',
                                                                              "you've",
                                                                              'won',
                                                                              'those',
                                                                              'from',
                                                                              'all',
                                                                              "shouldn't",
                                                                              'to',
                                                                              'mightn', ...]),
                                                  'lemmatized_text')])),
                ('model',
                 PassiveAggressiveClassifier(C=0.12050679650559393,
                                             max_iter=300, n_jobs=2))])

*Посмотрим на какие слова акцентирует все внимание эта модель*

In [ ]:
importancies_words = pd.DataFrame({
    'Слово': model.named_steps.preprocessor.get_feature_names_out(),
    'Коэффициент важности': model.named_steps.model.coef_.tolist()[0]
}).sort_values(by='Коэффициент важности', ascending=False).head(30)
importancies_words['Слово'] = importancies_words['Слово'].str.split('__').str[-1]

fig = go.Figure()
fig.add_bar(
    x=importancies_words['Коэффициент важности'],
    y=importancies_words['Слово'],
    orientation='h',
    textposition='outside'
)
fig.update_layout(
    width=700,
    height=900,
    title='Топ-30 слов по важности',
    xaxis_title='Коэффициент важности',
    yaxis_title='Слово',
    template='plotly_white',
    yaxis=dict(autorange='reversed')
)
fig.show()


- Эти слова и вправду могут быть использованы только в токсичных твитах!

_____
### BERT

*Уменьшаем датасет*

In [41]:
Xy = comments[['cleared_text', 'toxic']].sample(n=25000, random_state=RANDOM_STATE).reset_index(drop=True)
X = Xy.cleared_text
y = Xy.toxic

*Подкачиваем токенизатор*

In [42]:
pretrained_weights = 'unitary/toxic-bert'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
model = BertModel.from_pretrained(pretrained_weights)

*Распределение длины токенов*

In [43]:
token_lengths = [len(tokenizer.encode(txt, max_length=512, truncation=True)) for txt in X]
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=token_lengths,
    nbinsx=50,
    marker_color='blue',
    opacity=0.75
))
fig.update_layout(
    title='Распределение длины токенов',
    xaxis_title='Token count',
    yaxis_title='Частота',
    xaxis=dict(range=[0, 515]),
    template='plotly_white',
    width=800,
    height=500
)
fig.show()

- Так как мы ограничили размерность векторного пространства эмбеддингов до 512, то виден всплеск на этой отметке, говорящий о том, что есть немалое кол-во твитов, вошедших в сэмпл, длиной более 512 слов.

*Токенизация и паддинг*

In [44]:
tokenized_texts = X.apply(
    lambda x: tokenizer.encode(x, truncation=True, add_special_tokens=True)
)
max_len = max(len(tokens) for tokens in tokenized_texts)
padded_inputs = np.array([tokens + [0]*(max_len - len(tokens)) for tokens in tokenized_texts])
attention_masks = np.where(padded_inputs != 0, 1, 0)

*CPU или GPU*

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device);

*Получение эмбеддингов с помощью BERT на сэмпле*

In [ ]:
features_path = 'prepared_data/bert_features.npy'
labels_path = 'prepared_data/bert_labels.npy'
if os.path.exists(features_path):
    features = np.load(features_path)
    labels = np.load(labels_path)
else:
    batch_size = 10
    embeddings = []
    for i in tqdm(range((padded_inputs.shape[0] + batch_size - 1) // batch_size), desc='Вычисление эмбеддингов'):
        batch_input = torch.LongTensor(padded_inputs[batch_size*i:batch_size*(i+1)]).to(device)
        batch_mask = torch.LongTensor(attention_masks[batch_size*i:batch_size*(i+1)]).to(device)
        with torch.no_grad():
            outputs = model(batch_input, attention_mask=batch_mask)
        cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(cls_embeddings)
    features = np.concatenate(embeddings)
    labels = y.values
    np.save(features_path, features)
    np.save(labels_path, labels)

_____
### Вывод

_____
_____
## Общий вывод